# Zip Puzzle

## What is Zip Puzzle
In this pizzle you get a square board which certain cells have a number on them.

You should start from the cell that is marked as number 1 and then from there you can go Up, Down, Left, Right.

Here is an example:

![example Puzzle](./images/zip.png)

Here are the rules:
- You must cover all the cells on the board.
- You cannot enter any cell more than once.
- You must enter the numbered cells in order. For example, you cannot enter cell marked as 4 unless if you have entered first 1, then 2, and then 3. You can enter any un-numbered cells in between though.

Let's get into solving it.

NOTE: There are multiple method to do it. I liked to treat this as task scheduling, though.

## Importing packages and setting constants

In [1]:
from dataclasses import dataclass
from ortools.sat.python.cp_model import CpModel, CpSolver, IntVar, OPTIMAL
from tabulate import tabulate

In [2]:
@dataclass
class Cell:
    row_id: int
    col_id: int

    def is_horizontally_adjecent_to(self, cell: "Cell") -> bool:
        return (
            (self.row_id == cell.row_id) and 
            (abs(self.col_id -  cell.col_id) == 1)
        )

    def is_vertically_adjecent_to(self, cell: "Cell") -> bool:
        return (
            (self.col_id == cell.col_id) and 
            (abs(self.row_id -  cell.row_id) == 1)
        )

    def is_adjacent_to(self, cell: "Cell") -> bool:
        return self.is_horizontally_adjecent_to(cell) or self.is_vertically_adjecent_to(cell)

    def __hash__(self):
        return (self.row_id, self.col_id).__hash__()

In [3]:
@dataclass
class Barriers:
    def __init__(self):
        self._map: dict[Cell, set[cell]] = {}

    def add_barrier(self, cell_1: Cell, cell_2: Cell):
        if cell_1.is_adjacent_to(cell_2):
            self._map.setdefault(cell_1, set()).add(cell_2)
            self._map.setdefault(cell_2, set()).add(cell_1)
        else:
            raise ValueError("A barrier can be set only between two cells that are adjacent")
        
        return self
        
    def add_barriers(self, entries: list[tuple[Cell, Cell]]):
        for cell_1, cell_2 in entries:
            self.add_barrier(cell_1, cell_2)

        return self
            
    def has_barrier(self, cell_1: Cell, cell_2: Cell):
        return (
            (cell_1 in self._map) and (cell_2 in self._map[cell_1])
        ) or (
            (cell_2 in self._map) and (cell_1 in self._map[cell_2])
        )

In [4]:

    
class ZipSolver:
    DEFAULT_ROW_SIZE: int = 6
    DEFAULT_COL_SIZE: int = 6
    def __init__(
        self,
        barriers: Barriers = Barriers(),
        row_size: int = DEFAULT_ROW_SIZE,
        col_size: int = DEFAULT_COL_SIZE,
    ):
        self.model = CpModel()
        
        self.row_size: int = row_size
        self.col_size: int = col_size
        self.board_size: int = row_size * col_size

        self.model_tasks = []
        self.task_start_time = []
        self.task_end_time = []
        
        self._prepare()

        self.barriers = barriers

    def get_neighbor_cells(self, row_id: int, col_id: int) -> list[Cell]:
        neighbor_cells: list[Cell] = []

        # left 
        if col_id > 0 and not self.barriers.has_barrier(Cell(row_id, col_id), Cell(row_id, col_id - 1)):
            neighbor_cells.append(Cell(row_id, col_id - 1))

        # right
        if col_id < (self.col_size - 1) and not self.barriers.has_barrier(Cell(row_id, col_id), Cell(row_id, col_id + 1)):
            neighbor_cells.append(Cell(row_id, col_id + 1))

        # up
        if row_id > 0 and not self.barriers.has_barrier(Cell(row_id, col_id), Cell(row_id - 1, col_id)):
            neighbor_cells.append(Cell(row_id - 1, col_id))

        # down
        if row_id < (self.row_size - 1) and not self.barriers.has_barrier(Cell(row_id, col_id), Cell(row_id + 1, col_id)):
            neighbor_cells.append(Cell(row_id + 1, col_id))

        return neighbor_cells

    @staticmethod
    def move(cell_1: Cell, cell_2: Cell) -> str:
        if (cell_1.row_id == cell_2.row_id) and (cell_1.col_id == cell_2.col_id):
            raise ValueError("You need to move.")
            
        if cell_1.row_id == cell_2.row_id:
            return "Go Left" if cell_2.col_id < cell_1.col_id else "Go Right"

        if cell_1.col_id == cell_2.col_id:
            return "Go Up" if cell_2.row_id < cell_1.row_id else "Go Down"

        else:
            raise ValueError("Did you jump?!! No jumping is allowed.")

    def _prepare(self):
        self.task_start_time = [
            [
                self.model.NewIntVar(lb=0, ub = self.board_size, name=f"start_{i}_{j}")
                for j in range(self.col_size)
            ]
            for i in range(self.row_size)
        ]

        self.task_end_time = [
            [
                self.model.NewIntVar(lb=0, ub = self.board_size, name=f"end_{i}_{j}")
                for j in range(self.col_size)
            ]
            for i in range(self.row_size)
        ]
        self.model_tasks = [
            [
                self.model.NewIntervalVar(
                    start=self.task_start_time[i][j],
                    size=1,
                    end=self.task_end_time[i][j],
                    name=f"task_{i}_{j}",
                )
                for j in range(self.col_size)
            ]
            for i in range(self.row_size)
        ]

    def set_next_move(self, i: int, j: int):
        neighbor_cells = self.get_neighbor_cells(i,j)

        n_neighbors = len(neighbor_cells)

        bool_variables = [
            self.model.NewBoolVar(f"b_{i}_{j}_{idx}")
            for idx, neighbor_cell in enumerate(neighbor_cells)
        ]

        for idx, neighbor_cell in enumerate(neighbor_cells):
            self.model.Add(
                self.task_end_time[i][j] == self.task_start_time[neighbor_cell.row_id][neighbor_cell.col_id]
            ).OnlyEnforceIf(bool_variables[idx])

        # only one of the move must be taken
        self.model.add(sum(bool_variables) == 1)

    def generate_readable_solution(self):
        all_cells = sorted(
            [
                Cell(i,j)
                for i in range(self.row_size)
                for j in range(self.col_size)
            ],
            key=lambda c: self.solver.value(self.task_start_time[c.row_id][c.col_id])
        )

        
        print("  0: Start on:", all_cells[0])
        for i in range(1,self.board_size):
            print(f"{i:3d}: {self.move(all_cells[i - 1], all_cells[i]):<8} to {all_cells[i]}")
            
        

    def solve(self, cell_order: list[Cell], print_solution: bool = True):
        # First Cell starts first
        self.model.Add(self.task_start_time[cell_order[0].row_id][cell_order[0].col_id] == 0)

        # Last Cell ends the latest.
        self.model.Add(self.task_end_time[cell_order[-1].row_id][cell_order[-1].col_id] == self.board_size)

        for i in range(1,len(cell_order)):
            # forcing the previous cell in order to end before or on the start of next cell.
            self.model.Add(
                self.task_end_time[cell_order[i-1].row_id][cell_order[i-1].col_id] <=
                self.task_start_time[cell_order[i].row_id][cell_order[i].col_id]
            )

        # Now we need to set possible moves for each cell
        for i in range(self.row_size):
            for j in range(self.col_size):
                if (cell_order[-1].row_id == i) and (cell_order[-1].col_id == j):
                    # This is the last cell that we should end up. There is no more moves.
                    continue

                self.set_next_move(i,j)
                    

        self.solver = CpSolver()
        
        status = self.solver.solve(self.model)

        output = status == OPTIMAL
        if output and print_solution:
            self.generate_readable_solution()
            
        return output


### 2025-11-16

In [5]:
zip_solver = ZipSolver()

status = zip_solver.solve(cell_order=[
    Cell(2, 4),
    Cell(5, 4),
    Cell(4, 4),
    Cell(3, 4),
    Cell(1, 5),
    Cell(1, 4),
    Cell(1, 3),
    Cell(0, 2),
    Cell(0, 0),
    Cell(0, 1),
    Cell(1, 1),
    Cell(2, 1),
    Cell(3, 1),
    Cell(4, 1),
])

if status:
    print("Found a solution.")
else:
    print("Failed to find a solution.")

  0: Start on: Cell(row_id=2, col_id=4)
  1: Go Left  to Cell(row_id=2, col_id=3)
  2: Go Down  to Cell(row_id=3, col_id=3)
  3: Go Down  to Cell(row_id=4, col_id=3)
  4: Go Down  to Cell(row_id=5, col_id=3)
  5: Go Right to Cell(row_id=5, col_id=4)
  6: Go Right to Cell(row_id=5, col_id=5)
  7: Go Up    to Cell(row_id=4, col_id=5)
  8: Go Left  to Cell(row_id=4, col_id=4)
  9: Go Up    to Cell(row_id=3, col_id=4)
 10: Go Right to Cell(row_id=3, col_id=5)
 11: Go Up    to Cell(row_id=2, col_id=5)
 12: Go Up    to Cell(row_id=1, col_id=5)
 13: Go Up    to Cell(row_id=0, col_id=5)
 14: Go Left  to Cell(row_id=0, col_id=4)
 15: Go Down  to Cell(row_id=1, col_id=4)
 16: Go Left  to Cell(row_id=1, col_id=3)
 17: Go Up    to Cell(row_id=0, col_id=3)
 18: Go Left  to Cell(row_id=0, col_id=2)
 19: Go Down  to Cell(row_id=1, col_id=2)
 20: Go Down  to Cell(row_id=2, col_id=2)
 21: Go Down  to Cell(row_id=3, col_id=2)
 22: Go Down  to Cell(row_id=4, col_id=2)
 23: Go Down  to Cell(row_id=5, col_

### 2025-11-25

In [6]:
zip_solver = ZipSolver()

status = zip_solver.solve(cell_order=[
    Cell(4, 3),
    Cell(0, 3),
    Cell(4, 2),
    Cell(1, 2),
    Cell(3, 3),
    Cell(2, 3),
    Cell(2, 2),
])

if status:
    print("Found a solution.")
else:
    print("Failed to find a solution.")

  0: Start on: Cell(row_id=4, col_id=3)
  1: Go Right to Cell(row_id=4, col_id=4)
  2: Go Up    to Cell(row_id=3, col_id=4)
  3: Go Up    to Cell(row_id=2, col_id=4)
  4: Go Up    to Cell(row_id=1, col_id=4)
  5: Go Left  to Cell(row_id=1, col_id=3)
  6: Go Up    to Cell(row_id=0, col_id=3)
  7: Go Right to Cell(row_id=0, col_id=4)
  8: Go Right to Cell(row_id=0, col_id=5)
  9: Go Down  to Cell(row_id=1, col_id=5)
 10: Go Down  to Cell(row_id=2, col_id=5)
 11: Go Down  to Cell(row_id=3, col_id=5)
 12: Go Down  to Cell(row_id=4, col_id=5)
 13: Go Down  to Cell(row_id=5, col_id=5)
 14: Go Left  to Cell(row_id=5, col_id=4)
 15: Go Left  to Cell(row_id=5, col_id=3)
 16: Go Left  to Cell(row_id=5, col_id=2)
 17: Go Up    to Cell(row_id=4, col_id=2)
 18: Go Left  to Cell(row_id=4, col_id=1)
 19: Go Down  to Cell(row_id=5, col_id=1)
 20: Go Left  to Cell(row_id=5, col_id=0)
 21: Go Up    to Cell(row_id=4, col_id=0)
 22: Go Up    to Cell(row_id=3, col_id=0)
 23: Go Up    to Cell(row_id=2, col_

### 2025-11-26

In [7]:
zip_solver = ZipSolver(
    row_size=7,
    col_size=7,
    barriers=Barriers().add_barriers([
        (Cell(0, 1), Cell(1, 1)),
        (Cell(0, 2), Cell(1, 2)),
        (Cell(0, 3), Cell(1, 3)),
        (Cell(0, 4), Cell(1, 4)),
        (Cell(0, 5), Cell(1, 5)),
        (Cell(2, 5), Cell(3, 5)),
        (Cell(2, 1), Cell(3, 1)),
        (Cell(5, 3), Cell(6, 3)),
        (Cell(1, 0), Cell(1, 1)),
        (Cell(2, 0), Cell(2, 1)),
        (Cell(1, 5), Cell(1, 6)),
        (Cell(2, 5), Cell(2, 6)),
        (Cell(3, 1), Cell(3, 2)),
        (Cell(4, 1), Cell(4, 2)),
        (Cell(3, 4), Cell(3, 5)),
        (Cell(4, 4), Cell(4, 5)),
        (Cell(5, 2), Cell(5, 3)),
        (Cell(5, 3), Cell(5, 4)),
    ])
)

status = zip_solver.solve(cell_order=[
    Cell(5, 3), 
    Cell(3, 2),
    Cell(1, 3),
    Cell(3, 3),
    Cell(3, 4),
    Cell(2, 6),
    Cell(2, 0),
    Cell(4, 0),
    Cell(4, 6),
])

if status:
    print("Found a solution.")
else:
    print("Failed to find a solution.")

  0: Start on: Cell(row_id=5, col_id=3)
  1: Go Up    to Cell(row_id=4, col_id=3)
  2: Go Left  to Cell(row_id=4, col_id=2)
  3: Go Up    to Cell(row_id=3, col_id=2)
  4: Go Up    to Cell(row_id=2, col_id=2)
  5: Go Left  to Cell(row_id=2, col_id=1)
  6: Go Up    to Cell(row_id=1, col_id=1)
  7: Go Right to Cell(row_id=1, col_id=2)
  8: Go Right to Cell(row_id=1, col_id=3)
  9: Go Right to Cell(row_id=1, col_id=4)
 10: Go Right to Cell(row_id=1, col_id=5)
 11: Go Down  to Cell(row_id=2, col_id=5)
 12: Go Left  to Cell(row_id=2, col_id=4)
 13: Go Left  to Cell(row_id=2, col_id=3)
 14: Go Down  to Cell(row_id=3, col_id=3)
 15: Go Right to Cell(row_id=3, col_id=4)
 16: Go Down  to Cell(row_id=4, col_id=4)
 17: Go Down  to Cell(row_id=5, col_id=4)
 18: Go Right to Cell(row_id=5, col_id=5)
 19: Go Up    to Cell(row_id=4, col_id=5)
 20: Go Up    to Cell(row_id=3, col_id=5)
 21: Go Right to Cell(row_id=3, col_id=6)
 22: Go Up    to Cell(row_id=2, col_id=6)
 23: Go Up    to Cell(row_id=1, col_

### 2025-11-27

![another](./images/zip_02.png)

In [8]:
zip_solver = ZipSolver(
    barriers=Barriers().add_barriers([
        (Cell(0, 2), Cell(1, 2)),
        (Cell(0, 3), Cell(1, 3)),
        (Cell(1, 2), Cell(2, 2)),
        (Cell(1, 3), Cell(2, 3)),
        (Cell(2, 1), Cell(3, 1)),
        (Cell(2, 2), Cell(3, 2)),
        (Cell(2, 3), Cell(3, 3)),
        (Cell(2, 4), Cell(3, 4)),
        (Cell(4, 1), Cell(3, 1)),
        (Cell(4, 2), Cell(3, 2)),
        (Cell(4, 3), Cell(3, 3)),
        (Cell(4, 4), Cell(3, 4)),
    ])
)

status = zip_solver.solve(cell_order=[
    Cell(4, 2),
    Cell(4, 1),
    Cell(4, 0),
    Cell(4, 3),
    Cell(4, 4),
    Cell(4, 5),
    Cell(2, 4),
    Cell(2, 1),
    Cell(2, 2),
    Cell(2, 3),
])

if status:
    print("Found a solution.")
else:
    print("Failed to find a solution.")

  0: Start on: Cell(row_id=4, col_id=2)
  1: Go Left  to Cell(row_id=4, col_id=1)
  2: Go Left  to Cell(row_id=4, col_id=0)
  3: Go Down  to Cell(row_id=5, col_id=0)
  4: Go Right to Cell(row_id=5, col_id=1)
  5: Go Right to Cell(row_id=5, col_id=2)
  6: Go Right to Cell(row_id=5, col_id=3)
  7: Go Up    to Cell(row_id=4, col_id=3)
  8: Go Right to Cell(row_id=4, col_id=4)
  9: Go Down  to Cell(row_id=5, col_id=4)
 10: Go Right to Cell(row_id=5, col_id=5)
 11: Go Up    to Cell(row_id=4, col_id=5)
 12: Go Up    to Cell(row_id=3, col_id=5)
 13: Go Left  to Cell(row_id=3, col_id=4)
 14: Go Left  to Cell(row_id=3, col_id=3)
 15: Go Left  to Cell(row_id=3, col_id=2)
 16: Go Left  to Cell(row_id=3, col_id=1)
 17: Go Left  to Cell(row_id=3, col_id=0)
 18: Go Up    to Cell(row_id=2, col_id=0)
 19: Go Up    to Cell(row_id=1, col_id=0)
 20: Go Up    to Cell(row_id=0, col_id=0)
 21: Go Right to Cell(row_id=0, col_id=1)
 22: Go Right to Cell(row_id=0, col_id=2)
 23: Go Right to Cell(row_id=0, col_